In [1]:
!pip install -q unsloth
!pip install -q transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.2/46.2 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 192.7/192.7 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 24.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.1/162.1 kB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 68.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 40.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 45.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [4]:
from unsloth import FastModel


model, tokenizer = FastModel.from_pretrained(
    model_name="Jnaranjo/gemma-3-1b-grpo",
    max_seq_length=1024,
    load_in_4bit=False,
    load_in_8bit=False,
)

<ipython-input-4-11dcb11f1fdf>:1: UserWarning: WARNING: Unsloth should be imported before transformers, peft to ensure all optimizations are applied. Your code may run slower or encounter memory issues without these optimizations.

Please restructure your imports with 'import unsloth' at the top of your file.
  from unsloth import FastModel


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
Unsloth: Failed to patch Gemma3ForConditionalGeneration.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.3.19: Fast Gemma3 patching. Transformers: 4.51.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: Using float16 precision for gemma3 won't work! Using float32.
Unsloth: QLoRA and full finetuning all not selected. Switching to 16bit LoRA.


generation_config.json:   0%|          | 0.00/215 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.16M [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.69M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/33.4M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/35.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/26.1M [00:00<?, ?B/s]

In [10]:
import torch
import re
import random 
from unsloth import FastModel
from datasets import load_dataset
from tqdm.notebook import tqdm 


NUM_RANDOM_SAMPLES = 20
RANDOM_SEED = 42 
MODEL_NAME = "Jnaranjo/gemma-3-1b-grpo"
MAX_SEQ_LENGTH = 1024
LOAD_IN_4BIT = False
LOAD_IN_8BIT = False
MAX_NEW_TOKENS = 400
TEMPERATURE = 0.6
TOP_P = 0.9
TOP_K = 50


if RANDOM_SEED is not None:
    print(f"Setting random seed to: {RANDOM_SEED}")
    random.seed(RANDOM_SEED)


if not torch.cuda.is_available():
    print("WARNING: CUDA not available. Running on CPU will be very slow.")
    device = "cpu"
    load_in_4bit = False
    load_in_8bit = False
else:
    print("CUDA available. Using GPU.")
    device = "cuda"
    load_in_4bit = LOAD_IN_4BIT
    load_in_8bit = LOAD_IN_8BIT 

print(f"Loading model: {MODEL_NAME}...")
model, tokenizer = FastModel.from_pretrained(
    model_name=MODEL_NAME,
    max_seq_length=MAX_SEQ_LENGTH,
    load_in_4bit=load_in_4bit,
    load_in_8bit=load_in_8bit,
 
)
print("Model loaded.")


print("Loading GSM8K dataset...")
try:
    dataset = load_dataset("openai/gsm8k", "main")
    test_set = dataset['test']
    total_test_samples = len(test_set)
    print(f"Full test set size: {total_test_samples}")

    if NUM_RANDOM_SAMPLES > total_test_samples:
        print(f"Warning: Requested {NUM_RANDOM_SAMPLES} samples, but test set only has {total_test_samples}. Using all test samples.")
        NUM_RANDOM_SAMPLES = total_test_samples
        random_indices = list(range(total_test_samples))
    else:
  
        print(f"Selecting {NUM_RANDOM_SAMPLES} random samples from the test set...")
        random_indices = random.sample(range(total_test_samples), NUM_RANDOM_SAMPLES)
        print(f"Selected indices (first 5): {random_indices[:5]}...") 

    subset = test_set.select(random_indices)
    print(f"Created random subset of {len(subset)} questions.")

except Exception as e:
    print(f"Error loading dataset or creating subset: {e}")
    print("Please ensure you have internet connectivity and the 'datasets' library installed.")
    exit()


system_prompt = "Te presentan un problema. Piensa en el mismo y divide al mismo en pasos. Tus pasos deben ir dentro de <pensando> y </pensando>. Luego debes colocar la solución entre: <Solucion> y </Solucion>"



def extract_gsm8k_answer(answer_str):
    """Extracts the final numerical answer from the GSM8K ground truth string."""
    match = re.search(r"####\s*([\d\.,]+)", answer_str)
    if not match:
         match = re.search(r"\\boxed\{([\d\.,]+)\}", answer_str)
    if match:
        ans_str = match.group(1).replace(",", "").strip()
        try:
            return float(ans_str)
        except ValueError:
           
            return None
    else:

        return None

def extract_model_answer(model_output):
    """Extracts the numerical answer from the model's output within <Solucion> tags."""
    match = re.search(r"<Solucion>(.*?)</Solucion>", model_output, re.IGNORECASE | re.DOTALL)
    if match:
        extracted_text = match.group(1).strip()
        num_match = re.findall(r"[\d\.,]+", extracted_text)
        if num_match:
            ans_str = num_match[-1].replace(",", "").strip()
            try:
                return float(ans_str)
            except ValueError:
            
                return None
        else:
             
             return None
    else:
        num_match = re.findall(r"[\d\.,]+(?=\D*$)", model_output)
        if not num_match:
             num_match = re.findall(r"[\d\.,]+", model_output)
        if num_match:
            ans_str = num_match[-1].replace(",", "").strip()
            try:
                
                 return float(ans_str)
            except ValueError:
             
                 return None
        else:
           
            return None


results = []
correct_count = 0

print(f"\nStarting evaluation on {len(subset)} random questions...")
for i, item in enumerate(tqdm(subset, desc="Evaluating Random Questions")):
    question_index = random_indices[i] 
    question = item['question']
    true_answer_str = item['answer']
    true_answer_val = extract_gsm8k_answer(true_answer_str)

    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": question},
    ]


    try:
        text = tokenizer.apply_chat_template(
            messages,
            add_generation_prompt=True,
            tokenize=False,
        )
    except Exception as e:
        print(f"\nError applying chat template for question index {question_index}: {e}")
        results.append({
            "question_index": question_index, "question": question, "true_answer_val": true_answer_val,
            "model_output": "Error in tokenization", "model_answer_val": None, "correct": False
        })
        continue

    inputs = tokenizer(text, return_tensors="pt").to(device)


    generated_output = ""
    model_answer_val = None
    try:
        with torch.no_grad():
            outputs = model.generate(
                **inputs,
                max_new_tokens=MAX_NEW_TOKENS,
                temperature=TEMPERATURE,
                top_p=TOP_P,
                top_k=TOP_K,
                do_sample=True,
                pad_token_id=tokenizer.eos_token_id
            )
        input_length = inputs.input_ids.shape[1]
        generated_output = tokenizer.decode(outputs[0][input_length:], skip_special_tokens=True)
        model_answer_val = extract_model_answer(generated_output)

    except Exception as e:
        print(f"\nError during model generation or decoding for question index {question_index}: {e}")
        generated_output = f"Error during generation: {e}"

    is_correct = False
    if true_answer_val is not None and model_answer_val is not None:
        if abs(true_answer_val - model_answer_val) < 1e-4:
            is_correct = True

    if is_correct:
        correct_count += 1

    results.append({
        "question_index": question_index, 
        "question": question,
        "true_answer_val": true_answer_val,
        "model_output": generated_output.strip(),
        "model_answer_val": model_answer_val,
        "correct": is_correct,
    })


print("\n--- Evaluation Results (Random Sample) ---")


for res in results:
    print(f"\nQuestion Index: {res['question_index']}")
    print(f"Question: {res['question']}")
    print(f"Ground Truth Answer: {res['true_answer_val']}")
    print(f"Model Output:\n{res['model_output']}")
    print(f"Extracted Model Answer: {res['model_answer_val']}")
    print(f"Result: {'CORRECT' if res['correct'] else 'INCORRECT'}")
    print("-" * 30)

accuracy = (correct_count / len(subset)) * 100
print(f"\n--- Summary ---")
print(f"Total Questions Evaluated (Random Sample): {len(subset)}")
print(f"Correct Answers: {correct_count}")
print(f"Accuracy: {accuracy:.2f}%")


Setting random seed to: 42
CUDA available. Using GPU.
Loading model: Jnaranjo/gemma-3-1b-grpo...
==((====))==  Unsloth 2025.3.19: Fast Gemma3 patching. Transformers: 4.51.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: Using float16 precision for gemma3 won't work! Using float32.
Unsloth: QLoRA and full finetuning all not selected. Switching to 16bit LoRA.
Model loaded.
Loading GSM8K dataset...
Full test set size: 1319
Selecting 20 random samples from the test set...
Selected indices (first 5): [1309, 228, 51, 563, 501]...
Created random subset of 20 questions.

Starting evaluation on 20 random questions...


Evaluating Random Questions:   0%|          | 0/20 [00:00<?, ?it/s]


--- Evaluation Results (Random Sample) ---

Question Index: 1309
Question: The girls are trying to raise money for a carnival. Kim raises $320 more than Alexandra, who raises $430, and Maryam raises $400 more than Sarah, who raises $300. How much money, in dollars, did they all raise in total?
Ground Truth Answer: 2280.0
Model Output:
<pensando>Let K be the amount Kim raised, A be the amount Alexandra raised, M be the amount Maryam raised, and S be the amount Sarah raised.
We are given that Kim raises $320 more than Alexandra, so K = A + 320.
Alexandra raises $430, so A = 430.
Maryam raises $400 more than Sarah, so M = S + 400.
Sarah raises $300, so S = 300.
We want to find the total amount raised by all three girls, which is K + A + M + S.
We can express K in terms of A: K = A + 320.
We can express M in terms of S: M = S + 400.
Now we can substitute the values of A and S into the expression for the total amount raised:
Total = K + A + M + S = (A + 320) + 430 + (S + 400) + 300
Total =